<a href="https://colab.research.google.com/github/andonyns/air-quality/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 01

## Grupo 04
- Jorge Ignacio Chavarría Herrera - B82073
- Antonio Badilla-Olivas - B80874
- Enrique Guillermo Vílchez Lizano - C18477
- Andony Nuñez Solano - B04539

## Objetivos

1. Selección y recolección de parámetros y ciudades.
2. Limpiar y transformar los datos para comparaciones.
3. Análisis univariable y multivariable. Analizar las tendencias de los indicadores y hacer comparaciones. Incluir posibles correlaciones entre variables.
4. Conclusiones y recomendaciones según las políticas ambientales de cada país.

# Relevant concepts
These are the variables that [OpenAQ](https://openaq.org/) offers to measure air pollution. The definitions were taken from [Clean Air Act](https://www.epa.gov/criteria-air-pollutants/information-pollutant):

1. PM (Particular Matter): These particles come in many sizes and shapes and can be made up of hundreds of different chemicals. Some are emitted directly from a source, such as construction sites, unpaved roads, fields, smokestacks or fires. Most particles form in the atmosphere as a result of complex reactions of chemicals such as sulfur dioxide and nitrogen oxides, which are pollutants emitted from power plants, industries and automobiles.

  - PM₂.₅ (Particulate Matter 2.5 micrometers or smaller):
fine inhalable particles, with diameters that are generally 2.5 micrometers and smaller

  - PM₁₀ (Particulate Matter 10 micrometers or smaller):
inhalable particles, with diameters that are generally 10 micrometers and smaller

2.	O₃ (Ozone):
tropospheric, or ground level ozone, is not emitted directly into the air, but is created by chemical reactions between oxides of nitrogen (NOx) and volatile organic compounds (VOC). This happens when pollutants emitted by cars, power plants, industrial boilers, refineries, chemical plants, and other sources chemically react in the presence of sunlight.

3.	NO₂ (Nitrogen Dioxide):
Nitrogen Dioxide (NO2) is one of a group of highly reactive gases known as oxides of nitrogen or nitrogen oxides (NOx). Other nitrogen oxides include nitrous acid and nitric acid. NO2 is used as the indicator for the larger group of nitrogen oxides. NO2 primarily gets in the air from the burning of fuel. NO2 forms from emissions from cars, trucks and buses, power plants, and off-road equipment.

4.	SO₂ (Sulfur Dioxide):
SO2 is the component of greatest concern and is used as the indicator for the larger group of gaseous sulfur oxides (SOx).  Other gaseous SOx (such as SO3) are found in the atmosphere at concentrations much lower than SO2. The largest source of SO2 in the atmosphere is the burning of fossil fuels by power plants and other industrial facilities. Smaller sources of SO2 emissions include: industrial processes such as extracting metal from ore; natural sources such as volcanoes; and locomotives, ships and other vehicles and heavy equipment that burn fuel with a high sulfur content.

5.	CO (Carbon Monoxide):
CO is a colorless, odorless gas that can be harmful when inhaled in large amounts. CO is released when something is burned. The greatest sources of CO to outdoor air are cars, trucks and other vehicles or machinery that burn fossil fuels. A variety of items in your home such as unvented kerosene and gas space heaters, leaking chimneys and furnaces, and gas stoves also release CO and can affect air quality indoors.

In [12]:
%pip install dotenv
%pip install openaq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# For API requests
import requests
from urllib.parse import urljoin
from openaq import OpenAQ

# For env
import os
from dotenv import load_dotenv

# For data manipulation
from pprint import pprint
import pandas as pd
import pickle as pkl

import time
import logging
import sys

from dataclasses import dataclass
import re


In [113]:
def set_color(text: str, color: str = "green") -> str:
    """
    Set the color of the text.
    :param text: The text to color.
    :param color: The color to set.
    :return: The colored text.
    """
    colors = {
        "green": "\033[32m",
        "yellow": "\033[33m",
        "red": "\033[31m",
        "blue": "\033[34m",
        "orange": "\033[38;5;214m",
        "reset": "\033[0m",
    }
    return f"{colors[color]}{text}{colors['reset']}"

class StdOutLogger(logging.Logger):
    def __init__(self):
        super().__init__(__name__)
        self.logger = logging.getLogger(__name__)
        # To log events
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S',
            stream=sys.stdout
        )

    def info(self, message: str):
        self.logger.info(set_color(message, "blue"))
        
    def success(self, message: str):
        self.logger.info(set_color(message, "green"))

    def warning(self, message: str):
        self.logger.warning(set_color(message, "orange"))

    def critical(self, message: str):
        self.logger.critical(set_color(message, "red"))


In [114]:
# Load logger
logger = StdOutLogger()

# Load environment variables
load_dotenv()

True

In [93]:
DATA_DIR = "data"
if not os.path.isdir(DATA_DIR):
    os.mkdir(DATA_DIR)

BASE_API_URL = "https://api.openaq.org/v3/"

HEADERS = {"X-API-Key": os.getenv("API_KEY")}

LOCATIONS_ENDPOINT = urljoin(BASE_API_URL, "locations/{location_id}")
MEASUREMENTS_ENDPOINT = urljoin(BASE_API_URL, "sensors/{sensor_id}/measurements/daily")

In [94]:
def fetch_data(
    base_url: str,
    headers: dict[str, str] | None = None,
    parameters: dict[str, any] | None = None,
    query_parameters: dict[str, any] | None = None,
    verbose: bool = False
):
    """
    Fetch data from the OpenAQ API.
    :param base_url: The base URL for the API endpoint.
    :param headers: Optional headers to include in the request.
    :param parameters: Optional parameters to format the URL.
    :param query_parameters: Optional query parameters to include in the request.
    :param verbose: If True, print the response headers.
    :return: The JSON response from the API.
    """

    
    if parameters is not None:
        base_url = base_url.format(**parameters)

    if query_parameters is not None:
        base_url = urljoin(base_url, "?" + "&".join(f"{k}={v}" for k, v in query_parameters.items()))

    if headers is not None:
        response = requests.get(
            url=base_url,
            headers=headers,
        )
    else:
        response = requests.get(
            url=base_url,
        )

    if response.status_code != 200:
        raise Exception(
            f"Request failed with status: {response.status_code}. Reason: {response.text}"
        )
    if verbose:
        pprint(response.headers)
    return response

In [95]:
# Costa Rica example
cr_location_id = 3070644

cr_location_data = fetch_data(
    base_url=LOCATIONS_ENDPOINT,
    headers=HEADERS,
    parameters={"location_id": cr_location_id},
).json()
pprint(cr_location_data)

{'meta': {'found': 1,
          'limit': 100,
          'name': 'openaq-api',
          'page': 1,
          'website': '/'},
 'results': [{'bounds': [-84.0417, 9.938, -84.0417, 9.938],
              'coordinates': {'latitude': 9.938, 'longitude': -84.0417},
              'country': {'code': 'CR', 'id': 29, 'name': 'Costa Rica'},
              'datetimeFirst': {'local': '2024-09-19T14:01:34-06:00',
                                'utc': '2024-09-19T20:01:34Z'},
              'datetimeLast': {'local': '2025-04-20T15:54:21-06:00',
                               'utc': '2025-04-20T21:54:21Z'},
              'distance': None,
              'id': 3070644,
              'instruments': [{'id': 4, 'name': 'Clarity Sensor'}],
              'isMobile': False,
              'isMonitor': False,
              'licenses': [{'attribution': {'name': 'Clarity', 'url': None},
                            'dateFrom': '2021-10-20',
                            'dateTo': None,
                            'id

In [26]:
# Get the first sensor
sensor = cr_location_data["results"][0]["sensors"][0]
not_finished = True
query_params = {"limit": 1000, "page": 1}

while not_finished: 
    sensor_id = sensor["id"]
    sensor_data = fetch_data(
        base_url=MEASUREMENTS_ENDPOINT,
        headers=HEADERS,
        parameters={"sensor_id": sensor_id},
        query_parameters=query_params,
    ).json()
    if len(sensor_data["results"]) < 1000:
        print(f"last page: {sensor_data["meta"]["page"]}")
        not_finished = False

    else:
        query_params["page"] += 1

print(f"Total pages: {query_params["page"]}")

last page: 13
Total pages: 13


In [115]:
def fetch_location_sensors_data(
    location_id: int,
):
    logger.info(f"Fetching data for location ID: {location_id}...")
    location_data = fetch_data(
        base_url=LOCATIONS_ENDPOINT,
        headers=HEADERS,
        parameters={"location_id": location_id},
    ).json()

    logger.success(f"Finished fetch for data of location ID: {location_id}")

    # Get all sensors info
    sensors_data = {}
    

    # Iterate through all sensors
    for i, sensor in enumerate(location_data["results"][0]["sensors"]):
        query_params = {"limit": 100, "page": 1}
        logger.info(f"Processing sensor number {i}")
        sensor_id = sensor["id"]
        sensors_data[sensor_id] = []

        not_finished = True
        while not_finished:


            sensor_response = fetch_data(
                base_url=MEASUREMENTS_ENDPOINT,
                headers=HEADERS,
                parameters={"sensor_id": sensor_id},
                query_parameters=query_params,
            )

            sensor_data = sensor_response.json()

            logger.info(f"Processing sensor number {i}, page {sensor_data["meta"]["page"]}")
            logger.warning(f"Remaining requests: {sensor_response.headers["X-Ratelimit-Remaining"]}")
            
            sensors_data[sensor_id].append(sensor_data)  

            if len(sensor_data["results"]) < 100:
                not_finished = False
            else:
                query_params["page"] += 1
            
            if (sensor_response.headers["X-Ratelimit-Remaining"] == "1"):
                logger.warning(f"Reached rate limit. Waiting for {sensor_response.headers["X-Ratelimit-Reset"]} seconds.")
                time.sleep(int(sensor_response.headers["X-Ratelimit-Reset"]))

        logger.success(f"Finished processing sensor number {i}")

    logger.success(f"Finished processing all sensors for location ID: {location_id}")
    return sensors_data

In [106]:
class Location:
    def __init__(self, country: str, name: str, id: int, sensors_data: dict = None):
        self.country: str = country
        self.name: str = name
        self.id: int = id
        self.sensors_data: dict = sensors_data

    def pull_sensors_data(self):
        self.sensors_data = fetch_location_sensors_data(self.id)
    
    @staticmethod
    def save(location: "Location"):
        if not os.path.isdir(DATA_DIR + f"/{location.country}_{location.name}"):
            os.mkdir(DATA_DIR + f"/{location.country}_{location.name}")

        with open(f"{DATA_DIR}/{location.country}_{location.name}/{location.country}_{location.name}.pkl", "wb") as f:
            pkl.dump(location, f)

    @staticmethod
    def load(location: "Location") -> "Location":
        with open(f"{DATA_DIR}/{location.country}_{location.name}/{location.country}_{location.name}.pkl", "rb") as f:
            location = pkl.load(f)
        
        return location


In [116]:
locations = [
    Location(country="Costa Rica", name="NASA GSFC Rutgers Calib. N13", id=3070644),
    Location(country="Spain", name="Escaldes-Engordany", id=9742)
]

In [117]:
# For every location, save all info
for location in locations:
    location.pull_sensors_data()
    Location.save(location)
    
    del location.sensors_data
    
    logger.success(f"Saved data for location {location.name}")

2025-04-20 22:59:26 - __main__ - INFO - Fetching data for location ID: 3070644...
2025-04-20 22:59:26 - __main__ - INFO - Finished fetch for data of location ID: 3070644
2025-04-20 22:59:26 - __main__ - INFO - Processing sensor number 0
2025-04-20 22:59:26 - __main__ - INFO - Processing sensor number 0, page 1
2025-04-20 22:59:26 - __main__ - WARNING - Remaining requests: 58
2025-04-20 22:59:27 - __main__ - INFO - Processing sensor number 0, page 2
2025-04-20 22:59:27 - __main__ - WARNING - Remaining requests: 57
2025-04-20 22:59:27 - __main__ - INFO - Finished processing sensor number 0
2025-04-20 22:59:27 - __main__ - INFO - Finished processing all sensors for location ID: 3070644
2025-04-20 22:59:27 - __main__ - INFO - Saved data for location NASA GSFC Rutgers Calib. N13
2025-04-20 22:59:27 - __main__ - INFO - Fetching data for location ID: 9742...
2025-04-20 22:59:27 - __main__ - INFO - Finished fetch for data of location ID: 9742
2025-04-20 22:59:27 - __main__ - INFO - Processing 

KeyboardInterrupt: 


# 2. Tareas de limpieza y transformación:

Se deben realizar las tareas de limpieza y transformación necesarias para poder hacer un comparativo de la evolución de los diferentes indicadores de la calidad del aire en Costa Rica y las otras ciudades.


### Eliminar datos no necesarios

Primero, dejamos solo datos relevantes para analizar. Es decir, summary, value, parameter y period.

In [76]:
def clean_sensors_data(
    sensors_data: dict[int, list[dict]]
) -> dict[int, list[dict]]:
    
    """
    Clean the sensors data.
    :param sensors_data: The sensors data to clean.
    :return: The cleaned sensors data.
    """

    cleaned_data = {}
    for sensor_id, sensor_data in sensors_data.items():
        cleaned_data[sensor_id] = []

        for page in sensor_data:
            for measurement in page["results"]:
                # Data dict
                data_dict = {
                    "value": measurement["value"],
                    "summary": measurement["summary"],
                    "period": measurement["period"],
                    "parameter": measurement["parameter"],
                }
                cleaned_data[sensor_id].append(data_dict)
    
    return cleaned_data

In [121]:
# Load the data
locations = [Location.load(location) for location in locations]

for location in locations:
    location.sensors_data = clean_sensors_data(location.sensors_data)

### Buscamos valores nulos

Luego, buscamos valores nulos que puedan interferir en el análisis.

In [ ]:

def normalize_path(path):
    return re.sub(r'\[\d+\]', '[]', path)

def find_nulls(obj, path="", parent=None):
    nulls = []

    if isinstance(obj, dict):
        for k, v in obj.items():
            full_path = f"{path}.{k}" if path else k
            nulls.extend(find_nulls(v, full_path, obj))
    elif isinstance(obj, list):
        for i, item in enumerate(obj):
            full_path = f"{path}[{i}]"
            nulls.extend(find_nulls(item, full_path, item))
    elif obj is None:
        nulls.append((path, obj, parent))

    return nulls

seen_paths = set()

for location in locations:
    logger.info(f"Location: {location.name}")
    nulls = find_nulls(location.sensors_data)

    for path, val, parent in nulls:
        generalized = normalize_path(path)
        if generalized not in seen_paths:
            logger.warning(f"Null found at: {path}")
            logger.info(f"   Parent data: {parent}")
            seen_paths.add(generalized)

2025-04-20 23:03:53 - __main__ - INFO - Location: NASA GSFC Rutgers Calib. N13
2025-04-20 23:03:53 - __main__ - WARNING - Null found at: 10669679[0].parameter.displayName
2025-04-20 23:03:53 - __main__ - INFO -    Parent data: {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': None}
2025-04-20 23:03:53 - __main__ - INFO - Location: Escaldes-Engordany
2025-04-20 23:03:53 - __main__ - WARNING - Null found at: 30389[0].parameter.displayName
2025-04-20 23:03:53 - __main__ - INFO -    Parent data: {'id': 4, 'name': 'co', 'units': 'µg/m³', 'displayName': None}
2025-04-20 23:03:53 - __main__ - WARNING - Null found at: 30389[55].summary.sd
2025-04-20 23:03:53 - __main__ - INFO -    Parent data: {'min': 200.0, 'q02': 200.0, 'q25': 200.0, 'median': 200.0, 'q75': 200.0, 'q98': 200.0, 'max': 200.0, 'avg': 200.0, 'sd': None}
2025-04-20 23:03:53 - __main__ - WARNING - Null found at: 4272958[0].parameter.displayName
2025-04-20 23:03:53 - __main__ - INFO -    Parent data: {'id': 19843, 'name':

Parece que los valores nulos son los nombres de los parámetros (que no son muy importantes), y las desviaciones estándar de algunos valores agregados (pero solo aquellos donde solo existe un valor, lo cual tiene sentido). 


# 3. Implementación en Google Colab:

Realizar la implementación en Google Colab. Si existen problemas de desempeño, se puede optar por otro entorno, lo cual debe ser anotado en la documentación del notebook así como en la presentación.



# 4. Análisis y comparación:

Se debe realizar un análisis EDA que incluya análisis univariable y multivariable.

Analizar las tendencias de los indicadores para las diferentes ciudades y hacer comparaciones entre diferentes países y ciudades.

Incluir posibles correlaciones entre las variables y parámetros de calidad del aire de cada país/ciudad.

Utilizar diferentes tipos de visualizaciones relevantes para el análisis.



# 5. Conclusiones y Recomendaciones:

Extraer conclusiones sobre la evolución de la calidad del aire en Costa Rica y las ciudades seleccionadas, explicando cómo los datos sustentan estas conclusiones.

Buscar información sobre las políticas ambientales y regulaciones en estas ciudades y mostrar cómo los datos reflejan el efecto de estas políticas.